## Loading Libraries and Data

In [ ]:

from os import walk

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from keras import callbacks
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
sample_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
X_train = np.array(train.drop(['label'],axis=1)).reshape(len(train),28,28)
X_train.shape

In [ ]:
y_train = np.array(train['label']).reshape(len(train),1)
y_train.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
X_train, y_train, test_size=0.20, random_state=42)

In [ ]:
plt.imshow(X_val[0])
print(y_val[0])

In [ ]:
X_test = np.array(test).reshape(len(test),28,28)
X_test.shape

## Noramlizing Data

In [ ]:
X_train = X_train/255.0
X_val = X_val/255.0
X_test = X_test/255.0

## Model Architecture

In [ ]:
def my_model():
    inputs = keras.layers.Input(shape=(28,28,1))
    x = keras.layers.ZeroPadding2D(padding=(1,1),input_shape=(28,28,1))(inputs)
    x = keras.layers.Conv2D(32,3)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = keras.layers.MaxPooling2D(pool_size=2,strides=2)(x)
    x = keras.layers.ZeroPadding2D(padding=(1,1))(x)
    x = keras.layers.Dropout(0.1)(x)

    
    x = keras.layers.Conv2D(64,3)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = keras.layers.MaxPooling2D(pool_size=2,strides=2)(x)
    x = keras.layers.Dropout(0.2)(x)
    

    x = keras.layers.Conv2D(256,3)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = keras.layers.MaxPooling2D(pool_size=2,strides=2)(x)
    x = keras.layers.Dropout(0.3)(x)
    
    
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(128,activation='relu')(x)
    outputs = keras.layers.Dense(10,activation='softmax')(x)
    x = keras.layers.Dropout(0.4)(x)
    
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model
    
    
    
    
model = my_model()
model.summary()

## Callbacks

- Saving the best model (with the lowest loss)
- Learning Rate Decay at 25th, 50th and 75th Epoch


In [ ]:
### CallBacks

count = input('count: ')

### Save the best model
saved_callbacks = callbacks.ModelCheckpoint('./saved_models/bestmodel_'+str(count)+' '+ str(message),
                                     save_weights_only=False, 
                                     monitor='loss', 
                                     save_best_only=True)

### Learning Rate Scheduler
def scheduler(epoch, lr):
    if epoch == 25:
        print('\n learning_rate:', lr/10)
        return lr / 10

    elif epoch == 50:
        print('\n learning_rate:', lr/10)
        return lr / 10
    
    elif epoch == 75:
        print('\n learning_rate:', lr/100)
        return lr / 100
    
    elif epoch == 80:
        print('\n learning_rate:', lr/2.5)
        return lr / 2.5

    else:
        return lr

    
### Complie and Fit Model 
model.compile(optimizer = keras.optimizers.Adam(),
             loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy',])  

history = model.fit(X_train, y_train, epochs=100, verbose=2,
          batch_size=64,
           callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler),
                     saved_callbacks]
    )



## Load the Best Model Version and Evaluate on Validation Data

In [ ]:
model = keras.models.load_model('./saved_models/bestmodel_'+str(count)+' '+ str(message))

In [ ]:
model.evaluate(X_val, y_val)

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
preds = pd.Series([np.argmax(i) for i in y_preds], name='Label')
preds.unique()

In [ ]:
plt.imshow(X_test[0])
print(preds[0])

## Create a submission file

In [ ]:
index_list = []
for i, item in enumerate(preds):
    index_list.append(i+1)
    
image_id = pd.Series(index_list, name='ImageId')
submit = pd.concat([image_id, preds], axis=1)

In [ ]:
submit.to_csv('./submission_'+str(count)+'.csv', index=False)

### Ensembling the csv's together (Mode of prediction)

In [ ]:
count = 2
csv_lst = [pd.read_csv('./submission_'+ str(i) +'.csv') for i in range(count+1)]

Label_df = pd.DataFrame()
for i in range(len(csv_lst)):
    Label_df = pd.concat([Label_df,csv_lst[i].Label], axis=1)
    Label_df = Label_df.rename(columns={"Label": "Label"+str(i)})
    
mode_df = Label_df.mode(axis=1)

final = pd.DataFrame(submit['ImageId'])
final['Label'] = mode_df[0].astype(int)
final

In [ ]:
final.to_csv('./submission.csv', index=False)